# ChurnGuard Dataset Analysis & Integration
## Real Telecom Customer Churn Dataset - 7,043 Records

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Load your telecom dataset
df = pd.read_csv(r'D:\Windows Data\Downloads\archive (6)\WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(f"Dataset Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

In [ ]:
# Dataset Overview
print("=== CHURNGUARD DATASET ANALYSIS ===")
print(f"Total Customers: {len(df):,}")
print(f"Features: {len(df.columns)}")
print(f"Missing Values: {df.isnull().sum().sum()}")

# Churn Distribution
churn_dist = df['Churn'].value_counts()
churn_rate = (churn_dist['Yes'] / len(df)) * 100
print(f"\nChurn Rate: {churn_rate:.1f}%")
print(f"Churned Customers: {churn_dist['Yes']:,}")
print(f"Retained Customers: {churn_dist['No']:,}")

df.head()

In [ ]:
# ChurnGuard Feature Engineering (Same as Production Pipeline)
def churnguard_preprocessing(df):
    """ChurnGuard production preprocessing pipeline"""
    df_processed = df.copy()
    
    # Handle TotalCharges (convert to numeric)
    df_processed['TotalCharges'] = pd.to_numeric(df_processed['TotalCharges'], errors='coerce')
    df_processed['TotalCharges'].fillna(df_processed['MonthlyCharges'], inplace=True)
    
    # ChurnGuard Feature Engineering
    df_processed['tenure_group'] = pd.cut(df_processed['tenure'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
    df_processed['avg_monthly_charges'] = df_processed['TotalCharges'] / (df_processed['tenure'] + 1)
    
    # Service count (ChurnGuard metric)
    service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    df_processed['total_services'] = df_processed[service_cols].apply(
        lambda x: sum([1 for val in x if val == 'Yes']), axis=1
    )
    
    # Encode target
    df_processed['Churn_Binary'] = df_processed['Churn'].map({'Yes': 1, 'No': 0})
    
    return df_processed

# Apply ChurnGuard preprocessing
df_processed = churnguard_preprocessing(df)
print("ChurnGuard preprocessing completed!")
print(f"New features: tenure_group, avg_monthly_charges, total_services")
df_processed[['customerID', 'tenure', 'tenure_group', 'avg_monthly_charges', 'total_services', 'Churn']].head()

In [ ]:
# ChurnGuard Analytics Dashboard Preview
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('ChurnGuard Analytics Dashboard Preview', fontsize=16, fontweight='bold')

# 1. Churn Rate by Tenure Group
tenure_churn = df_processed.groupby('tenure_group')['Churn_Binary'].mean() * 100
tenure_churn.plot(kind='bar', ax=axes[0,0], color='#3B82F6')
axes[0,0].set_title('Churn Rate by Tenure Group')
axes[0,0].set_ylabel('Churn Rate (%)')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Monthly Charges Distribution
df_processed.boxplot(column='MonthlyCharges', by='Churn', ax=axes[0,1])
axes[0,1].set_title('Monthly Charges by Churn Status')
axes[0,1].set_xlabel('Churn Status')

# 3. Contract Type Impact
contract_churn = df_processed.groupby('Contract')['Churn_Binary'].mean() * 100
contract_churn.plot(kind='bar', ax=axes[1,0], color='#10B981')
axes[1,0].set_title('Churn Rate by Contract Type')
axes[1,0].set_ylabel('Churn Rate (%)')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Payment Method Impact
payment_churn = df_processed.groupby('PaymentMethod')['Churn_Binary'].mean() * 100
payment_churn.plot(kind='bar', ax=axes[1,1], color='#F59E0B')
axes[1,1].set_title('Churn Rate by Payment Method')
axes[1,1].set_ylabel('Churn Rate (%)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n=== KEY INSIGHTS FOR CHURNGUARD ===")
print(f"Highest Risk Tenure: {tenure_churn.idxmax()} ({tenure_churn.max():.1f}% churn rate)")
print(f"Highest Risk Contract: {contract_churn.idxmax()} ({contract_churn.max():.1f}% churn rate)")
print(f"Highest Risk Payment: {payment_churn.idxmax()} ({payment_churn.max():.1f}% churn rate)")

In [ ]:
# ChurnGuard ML Model Training (Production Pipeline)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

# Prepare features for ChurnGuard ML pipeline
def prepare_features(df):
    """ChurnGuard feature preparation"""
    # Select features
    feature_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'total_services', 'avg_monthly_charges']
    categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'InternetService', 
                       'Contract', 'PaperlessBilling', 'PaymentMethod']
    
    X = df[feature_cols].copy()
    
    # One-hot encode categorical variables
    for col in categorical_cols:
        if col in df.columns:
            dummies = pd.get_dummies(df[col], prefix=col)
            X = pd.concat([X, dummies], axis=1)
    
    y = df['Churn_Binary']
    return X, y

# Prepare data
X, y = prepare_features(df_processed)
print(f"Features prepared: {X.shape[1]} features, {len(X)} samples")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE (ChurnGuard balancing)
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {len(X_train_scaled)} samples (balanced with SMOTE)")
print(f"Test set: {len(X_test_scaled)} samples")

In [ ]:
# ChurnGuard Multi-Model Training (Production Models)
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss')
}

results = {}
print("=== CHURNGUARD MODEL PERFORMANCE ===")
print("-" * 50)

for name, model in models.items():
    # Train model
    model.fit(X_train_scaled, y_train_balanced)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    
    # Metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, y_prob)
    
    results[name] = {
        'f1_score': report['weighted avg']['f1-score'],
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'auc': auc
    }
    
    # Display results
    f1_score = results[name]['f1_score'] * 100
    auc_score = results[name]['auc'] * 100
    print(f"{name:15} | F1: {f1_score:5.1f}% | AUC: {auc_score:5.1f}%")

# Find best model
best_model_name = max(results.keys(), key=lambda k: results[k]['f1_score'])
print(f"\n🏆 Best Model: {best_model_name}")
print(f"F1 Score: {results[best_model_name]['f1_score']:.3f}")
print(f"AUC Score: {results[best_model_name]['auc']:.3f}")

# Validate ChurnGuard requirements
if results[best_model_name]['f1_score'] >= 0.8:
    print("✅ Model meets ChurnGuard F1 > 0.8 requirement")
else:
    print("⚠️ Model F1 score below 0.8 threshold")

In [ ]:
# ChurnGuard High-Risk Customer Identification
best_model = models[best_model_name]
risk_probabilities = best_model.predict_proba(X_test_scaled)[:, 1]

# Create risk segments
test_customers = X_test.copy()
test_customers['risk_score'] = risk_probabilities
test_customers['risk_level'] = pd.cut(risk_probabilities, 
                                     bins=[0, 0.3, 0.7, 1.0], 
                                     labels=['Low', 'Medium', 'High'])
test_customers['actual_churn'] = y_test.values

# High-risk customers (ChurnGuard priority list)
high_risk = test_customers[test_customers['risk_level'] == 'High'].sort_values('risk_score', ascending=False)

print("=== CHURNGUARD HIGH-RISK CUSTOMERS ===")
print(f"Total High-Risk Customers: {len(high_risk)}")
print(f"Accuracy on High-Risk: {(high_risk['actual_churn'] == 1).mean()*100:.1f}%")
print("\nTop 10 Highest Risk Customers:")
print("-" * 60)

for idx, (_, customer) in enumerate(high_risk.head(10).iterrows(), 1):
    risk_pct = customer['risk_score'] * 100
    actual = "CHURNED" if customer['actual_churn'] == 1 else "RETAINED"
    print(f"{idx:2d}. Risk: {risk_pct:5.1f}% | Tenure: {customer['tenure']:2.0f} | Charges: ${customer['MonthlyCharges']:5.1f} | {actual}")

# Risk distribution
risk_dist = test_customers['risk_level'].value_counts()
print(f"\nRisk Distribution:")
for level in ['Low', 'Medium', 'High']:
    count = risk_dist.get(level, 0)
    pct = (count / len(test_customers)) * 100
    print(f"{level:6} Risk: {count:4d} customers ({pct:4.1f}%)")

In [ ]:
# Save processed dataset for ChurnGuard deployment
output_path = r'e:\churnforgeprediction\data\telecom_churn.csv'
df.to_csv(output_path, index=False)
print(f"✅ Dataset saved to ChurnGuard: {output_path}")
print(f"📊 Ready for deployment: {len(df):,} customers, {df['Churn'].value_counts()['Yes']:,} churned")
print(f"🎯 Expected model performance: F1 > {results[best_model_name]['f1_score']:.1%}")

# ChurnGuard deployment summary
print("\n" + "="*60)
print("🚀 CHURNGUARD DEPLOYMENT READY")
print("="*60)
print(f"Dataset: {len(df):,} telecom customers")
print(f"Features: {len(df.columns)} columns")
print(f"Churn Rate: {(df['Churn'] == 'Yes').mean()*100:.1f}%")
print(f"Best Model: {best_model_name} (F1: {results[best_model_name]['f1_score']:.1%})")
print(f"High-Risk Customers: {len(high_risk)} identified")
print("\n✅ Ready for AWS deployment with your production credentials!")